In [2]:
from ncpol2sdpa import*
import scipy.io as sio
import numpy as np
from math import sqrt
import matplotlib.pyplot as plt 
import pandas as pd
import time

In [4]:
# Load stock-market data
load_path = 'setting6.mat'
load_data = sio.loadmat(load_path)
seq=flatten(load_data['seq_d0'].tolist())

# output 'stock-market.txt' for Julia script
f=open('stock-market.txt','w')
for ele in seq:
    f.write(str(ele)+'\n')
f.close()

In [4]:
# Load stock-market data
load_path = 'data/setting6.mat'
load_data = sio.loadmat(load_path)
seq=flatten(load_data['seq_d0'].tolist())

In [8]:
experiment=open('data/stock_time_PY.csv','w')
experiment=open('data/stock_time_PY.csv','a')
level = 1
# Experiments
for T in range(5,21):
    R=np.zeros(3)
    for r in range(3):
        
        #if (T>15)&(r>1):
        #    continue
        #else:
        start=np.random.randint(0,100,size=1)[0]
        Y=seq[start:start+T]

        # Decision Variables
        G = generate_operators("G", n_vars=1, hermitian=False, commutative=False)[0]
        Fdash = generate_operators("Fdash", n_vars=1, hermitian=False, commutative=False)[0]
        m = generate_operators("m", n_vars=T+1, hermitian=False, commutative=False)
        q = generate_operators("q", n_vars=T, hermitian=False, commutative=False)
        p = generate_operators("p", n_vars=T, hermitian=False, commutative=False)
        f = generate_operators("f", n_vars=T, hermitian=False, commutative=False)

        # Objective
        obj = sum((Y[i]-f[i])**2 for i in range(T)) + 0.01*sum(p[i]**2 for i in range(T)) + 0.01*sum(q[i]**2 for i in range(T)) 

        # Constraints
        ine1 = [f[i] - Fdash*m[i+1] - p[i] for i in range(T)]
        ine2 = [-f[i] + Fdash*m[i+1] + p[i] for i in range(T)]
        ine4 = [m[i+1] - G*m[i] - q[i] for i in range(T)]
        ine5 = [-m[i+1] + G*m[i] + q[i] for i in range(T)]

        ines = ine1+ine2+ine4+ine5

        # Solve the NCPO
        start = time.time()

        sdp = SdpRelaxation(variables = flatten([G,Fdash,f,p,m,q]), verbose = 2)
        sdp.get_relaxation(level, objective=obj, inequalities=ines)

        #if (T<15):
        sdp.solve(solver='mosek')
        #sdp.solve(solver='sdpa', solverparameters={"executable":"sdpa_gmp","executable": "C:\\Users\\zhouq\\Documents\\sdpa7-windows\\sdpa.exe"})
        end = time.time()
        R[r]=end-start
        #else:
            #continue

    #if (T<15):
    print(T,',',np.mean(R),",",np.std(R),file=experiment) #,sdp.n_vars
    #else:
    #    print(" , ,",sdp.n_vars,file=experiment)

experiment.close()

The problem has 23 noncommuting Hermitian variables
Calculating block structure...
Estimated number of SDP variables: 1127
Generating moment matrix...
Reduced number of SDP variables: 1127127 (done: 100.09%, ETA 00:00:-0.0)
Processing 20/20 constraints...
Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 1127            
  Cones                  : 0               
  Scalar variables       : 0               
  Matrix variables       : 1               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                  

  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 1127            
  Cones                  : 0               
  Scalar variables       : 0               
  Matrix variables       : 1               
  Integer variables      : 0               

Optimizer  - threads                : 4               
Optimizer  - solved problem         : the primal      
Optimizer  - Constraints            : 1127
Optimizer  - Cones                  : 0
Optimizer  - Scalar variables       : 0                 conic                  : 0               
Optimizer  - Semi-definite variables: 1                 scalarized             : 2278            
Factor     - setup time             : 0.03              dense det. time        : 0.00            
Factor     - ML order time          : 0.03              GP order time          : 0.00            
Factor     - nonzeros before factor : 6.36e+

Optimizer  - Scalar variables       : 0                 conic                  : 0               
Optimizer  - Semi-definite variables: 1                 scalarized             : 2278            
Factor     - setup time             : 0.03              dense det. time        : 0.00            
Factor     - ML order time          : 0.02              GP order time          : 0.00            
Factor     - nonzeros before factor : 6.36e+05          after factor           : 6.36e+05        
Factor     - dense dim.             : 0                 flops                  : 4.83e+08        
ITE PFEAS    DFEAS    GFEAS    PRSTATUS   POBJ              DOBJ              MU       TIME  
0   6.7e+01  1.0e+00  2.0e+00  0.00e+00   1.000000000e+00   0.000000000e+00   1.0e+00  0.05  
1   3.0e-02  4.4e-04  3.5e-02  -1.00e+00  1.596697387e+03   3.189831362e+03   4.4e-04  0.11  
2   6.2e-04  9.3e-06  3.6e-03  -7.31e-01  2.727674071e+04   6.473442050e+04   9.3e-06  0.17  
3   3.6e-06  5.4e-08  4.5e-04  -9.48

1   2.8e-02  4.0e-04  3.3e-02  -1.00e+00  1.787715269e+03   3.571867257e+03   4.0e-04  0.13  
2   5.9e-04  8.3e-06  3.4e-03  -7.31e-01  3.050976944e+04   7.240993771e+04   8.3e-06  0.17  
3   3.4e-06  4.8e-08  4.3e-04  -9.48e-01  1.044126360e+07   3.033263382e+07   4.8e-08  0.25  
4   3.9e-12  5.5e-14  3.1e-07  -1.00e+00  4.468880717e+12   1.294600771e+13   5.3e-14  0.31  
Optimizer terminated. Time: 0.31    

The problem has 27 noncommuting Hermitian variables
Calculating block structure...
Estimated number of SDP variables: 1539
Generating moment matrix...
Reduced number of SDP variables: 1539539 (done: 100.06%, ETA 00:00:-0.0)
Processing 24/24 constraints...
Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 1539            
  Cones                  : 0               
  Scalar variables       : 0               
  Matrix variables       : 1            

Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 1539            
  Cones                  : 0               
  Scalar variables       : 0               
  Matrix variables       : 1               
  Integer variables      : 0               

Optimizer  - threads                : 4               
Optimizer  - solved problem         : the primal      
Optimizer  - Constraints            : 1539
Optimizer  - Cones                  : 0
Optimizer  - Scalar variables       : 0                 conic                  : 0               
Optimizer  - Semi-definite variables: 1                 scalarized             : 3160            
Factor     - setup time             : 0.08              dense det. time        : 0.00            
Factor     - ML order time          : 0.05              GP order time          : 0.00            
Factor     - nonzeros before factor 

...

Processing 76/76 constraints...
Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 12719           
  Cones                  : 0               
  Scalar variables       : 0               
  Matrix variables       : 1               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.02    
Problem
  Name                   :                 
  Objective sense        : min             
  Type           

Processing 76/76 constraints...
Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 12719           
  Cones                  : 0               
  Scalar variables       : 0               
  Matrix variables       : 1               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.00    
Problem
  Name                   :                 
  Objective sense        : min             
  Type           

Processing 80/80 constraints...
Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 14027           
  Cones                  : 0               
  Scalar variables       : 0               
  Matrix variables       : 1               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.00    
Problem
  Name                   :                 
  Objective sense        : min             
  Type           

Processing 80/80 constraints...
Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 14027           
  Cones                  : 0               
  Scalar variables       : 0               
  Matrix variables       : 1               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.02    
Problem
  Name                   :                 
  Objective sense        : min             
  Type           

Processing 80/80 constraints...
Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 14027           
  Cones                  : 0               
  Scalar variables       : 0               
  Matrix variables       : 1               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.00    
Problem
  Name                   :                 
  Objective sense        : min             
  Type           